# MobiML GeoTrackNet Demo

Based on: https://github.com/CIA-Oceanix/GeoTrackNet (MIT Licensed, (c) 2018 Duong Nguyen)

As presented in Nguyen, D., Vadaine, R., Hajduch, G., Garello, R. (2022). GeoTrackNet - A Maritime Anomaly Detector Using Probabilistic Neural Network Representation of AIS Tracks and A Contrario Detection. In IEEE Transactions on Intelligent Transportation Systems, 23(6).


Using data from AISDK: http://web.ais.dk/aisdata/aisdk-2018-02.zip

*It is possible to further explore maritime traffic patterns with the TrAISformer (https://github.com/CIA-Oceanix/TrAISformer), which is used for vessel trajectory prediction. The TrAISformer can be trained with AIS data and the preprocessing steps are similar to those of GeoTrackNet. However, the TrAISformer is out of the scope of MobiML and is an optional extension for the user to explore.*

## Environments

### Preprocessing

It is recommended to perform the preprocessing steps with the MobiML environment.

### Model Training

Set up a dedicated GeoTrackNet environment (PY3GPU) to train the model as instructed by Nguyen et al. (2022).

## Preprocessing

In [ ]:
import numpy as np
import os
import sys
import pickle
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from datetime import datetime, timedelta

sys.path.append("../src")
from mobiml.datasets import AISDK
from mobiml.samplers import TemporalSplitter
from mobiml.preprocessing import (
    TrajectorySplitter,
    TrajectoryFilter,
    TrajectoryDownsampler,
)

In [ ]:
%pwd

In [ ]:
# AISDK dataset
LAT, LON, SOG, COG, NAME, SHIPTYPE, NAV_STT, TIMESTAMP, TRAJ_ID = list(range(9))

EPOCH = datetime(1970, 1, 1)

SOG_MIN = 2.0
SOG_MAX = 30.0  # SOG is truncated to 30.0 knots max

# Pkl filenames
pkl_filename_train = "aisdk_20180208_train.pkl"
pkl_filename_valid = "aisdk_20180208_valid.pkl"
pkl_filename_test = "aisdk_20180208_test.pkl"

# Path to csv files
data_path = "data/aisdk_20180208_sample/"
csv_filename = "aisdk_20180208_sample.csv"

# Output path
out_path = "data/aisdk_20180208_sample/"

### Loading data

In [ ]:
path = os.path.join(data_path, csv_filename)
print(f"{datetime.now()} Loading data from {path}")
aisdk = AISDK(path)  # you can specify a bounding box here to filter the area
LON_MIN, LAT_MIN, LON_MAX, LAT_MAX = aisdk.get_bounds()
print(
    f"Bounding box:\nmin_lon: {LON_MIN}\nmin_lat: {LAT_MIN}\nmax_lon: {LON_MAX}\nmax_lat: {LAT_MAX}"
)

### Cleaning

#### Remove missing values

In [ ]:
aisdk.df = aisdk.df.dropna()
aisdk.df

In [ ]:
print("After removing missing values we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])
print("Total number of vessels:", len(aisdk.df.traj_id.unique()))
print("Lat min: ", aisdk.df.y.min(), "Lat max: ", aisdk.df.y.max())
print("Lon min: ", aisdk.df.x.min(), "Lon max: ", aisdk.df.x.max())
print("Time min: ", aisdk.df.timestamp.min(), "Time max: ", aisdk.df.timestamp.max())

#### Remove 'Moored' and 'At anchor' AIS messages

In [ ]:
aisdk.df.drop(aisdk.df[(aisdk.df["nav_status"] == "Moored") | (aisdk.df["nav_status"] == "At anchor")].index, inplace=True)
print("After removing 'Moored' or 'At anchor' AIS messages we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Keep only 'Cargo', 'Tanker', 'Passenger' vessel types

In [ ]:
aisdk.df = aisdk.df[
    (aisdk.df["ship_type"] == "Cargo")
    | (aisdk.df["ship_type"] == "Tanker")
    | (aisdk.df["ship_type"] == "Passenger")
]
print("After keeping only 'Cargo', 'Tanker' or 'Passenger' AIS messages we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Split trajectories with observation gaps > 2 hrs

In [ ]:
aisdk = TrajectorySplitter(aisdk).split(observation_gap=timedelta(hours=2))
print("After splitting trajectories with observation gaps we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Drop trajectories with fewer than $Points_{min}$ locations

In [ ]:
aisdk = TrajectoryFilter(aisdk).filter_min_pts(min_pts=20)
print("After removing trajectories with too few points we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Drop speed outliers

In [ ]:
aisdk = TrajectoryFilter(aisdk).filter_speed(min_speed=SOG_MIN, max_speed=SOG_MAX)
print("After removing speed outliers by setting a minimum and maximum speed we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

In [ ]:
tc = aisdk.to_trajs() #  mpd.TrajectoryCollection(aisdk.df, "traj_id", t="timestamp", x="x", y="y")
traj_gdf = tc.to_traj_gdf()

We may also want to remove trajectories based on their overall average speed rather than the SOG values:

In [ ]:
for index, row in traj_gdf.iterrows():
    traj_gdf.loc[index, "speed_ok"] = (
        tc.trajectories[index].get_length()
        / tc.trajectories[index].get_duration().total_seconds()
        > 1.02889  # 2 knots
    )

In [ ]:
traj_gdf = traj_gdf[traj_gdf["speed_ok"] == True]

In [ ]:
aisdk.df = pd.merge(aisdk.df, traj_gdf["traj_id"], how="inner")
print("After removing speed outliers based on length and duration we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

### Training data preparation
#### Subsample AIS tracks 

In [ ]:
aisdk = TrajectoryDownsampler(aisdk).subsample(min_dt_sec=60)
print("After subsampling AIS tracks we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Temporal train/valid/test split

In [ ]:
aisdk = TemporalSplitter(aisdk).split_hr()
aisdk.df

In [ ]:
aisdk_train = aisdk.df[(aisdk.df["split"] == 1.0)]
aisdk_valid = aisdk.df[(aisdk.df["split"] == 2.0)]
aisdk_test = aisdk.df[(aisdk.df["split"] == 3.0)]

print("Total number of AIS messages: ", len(aisdk.df))
print("Number of msgs in the training set: ", len(aisdk_train))
print("Number of msgs in the validation set: ", len(aisdk_valid))
print("Number of msgs in the test set: ", len(aisdk_test))

In [ ]:
aisdk_train

In [ ]:
target_column_order=["y", "x", "speed", "direction", "Name", "ship_type", "nav_status", "timestamp", "traj_id"]
aisdk_train = aisdk_train[target_column_order].reset_index(drop=True)
aisdk_valid = aisdk_valid[target_column_order].reset_index(drop=True)
aisdk_test = aisdk_test[target_column_order].reset_index(drop=True)
aisdk_test

#### Format timestamp

In [ ]:
aisdk_train["timestamp"] = (aisdk_train["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_valid["timestamp"] = (aisdk_valid["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_test["timestamp"]  = (aisdk_test["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_test

#### Format to ndarrays

In [ ]:
aisdk_train = np.array(aisdk_train)
aisdk_valid = np.array(aisdk_valid)
aisdk_test = np.array(aisdk_test)

#### Merging into dict
Creating AIS tracks from the list of AIS messages. Each AIS track is formatted by a dictionary.

In [ ]:
print("Convert to dicts of vessel's tracks...")

def convert_tracks_to_dicts(tracks):
    d = dict()
    for v_msg in tqdm(tracks):
        mmsi = int(v_msg[TRAJ_ID])
        if not (mmsi in list(d.keys())):
            d[mmsi] = np.empty((0, 9))
        d[mmsi] = np.concatenate(
            (d[mmsi], np.expand_dims(v_msg[:9], 0)), axis=0
        )
    for key in tqdm(list(d.keys())):
        d[key] = np.array(
            sorted(d[key], key=lambda m_entry: m_entry[TIMESTAMP])
        )
    return d

Vs_train = convert_tracks_to_dicts(aisdk_train)
Vs_valid = convert_tracks_to_dicts(aisdk_valid)
Vs_test = convert_tracks_to_dicts(aisdk_test)

#### Normalisation

In [ ]:
print("Normalising data ...")

def normalize(d):
    for k in tqdm(list(d.keys())):
        v = d[k]
        v[:, LAT] = (v[:, LAT] - LAT_MIN) / (LAT_MAX - LAT_MIN)
        v[:, LON] = (v[:, LON] - LON_MIN) / (LON_MAX - LON_MIN)
        v[:, SOG][v[:, SOG] > SOG_MAX] = SOG_MAX
        v[:, SOG] = v[:, SOG] / SOG_MAX
        v[:, COG] = v[:, COG] / 360.0
    return d 

Vs_train = normalize(Vs_train)
Vs_valid = normalize(Vs_valid)
Vs_test = normalize(Vs_test)

In [ ]:
for filename, filedict in zip(
    [pkl_filename_train, pkl_filename_valid, pkl_filename_test],
    [Vs_train, Vs_valid, Vs_test],
):
    print("Writing to", os.path.join(out_path, filename))
    with open(os.path.join(out_path, filename), "wb") as f:
        pickle.dump(filedict, f)

## Model Training

From this point forward, it is recommended to execute the code with the [PY3GPU environment](https://github.com/CIA-Oceanix/GeoTrackNet/blob/master/requirements.yml), as set up by Nguyen et al. (2022).

### Setup

In [ ]:
import os

# AISDK dataset
LAT, LON, SOG, COG, NAME, SHIPTYPE, NAV_STT, TIMESTAMP, TRAJ_ID = list(range(9))

# Pkl filenames
pkl_filename_train = "aisdk_20180208_train.pkl"
pkl_filename_valid = "aisdk_20180208_valid.pkl"
pkl_filename_test = "aisdk_20180208_test.pkl"

data_path = "../examples/data/aisdk_20180208_sample/"
dataset_path = os.path.join(data_path, pkl_filename_train)

### Calculate AIS mean

In [ ]:
import numpy as np
import pickle
import tensorflow as tf

LAT_BINS = 100
LON_BINS = 200
SOG_BINS = 30
COG_BINS = 72


def sparse_AIS_to_dense(msgs_, num_timesteps, mmsis):
    def create_dense_vect(msg, lat_bins=100, lon_bins=200, sog_bins=30, cog_bins=72):
        lat, lon, sog, cog = msg[0], msg[1], msg[2], msg[3]
        data_dim = lat_bins + lon_bins + sog_bins + cog_bins
        dense_vect = np.zeros(data_dim)
        dense_vect[int(lat * lat_bins)] = 1.0
        dense_vect[int(lon * lon_bins) + lat_bins] = 1.0
        dense_vect[int(sog * sog_bins) + lat_bins + lon_bins] = 1.0
        dense_vect[int(cog * cog_bins) + lat_bins + lon_bins + sog_bins] = 1.0
        return dense_vect

    dense_msgs = []
    for msg in msgs_:
        dense_msgs.append(
            create_dense_vect(
                msg,
                lat_bins=LAT_BINS,
                lon_bins=LON_BINS,
                sog_bins=SOG_BINS,
                cog_bins=COG_BINS,
            )
        )
    dense_msgs = np.array(dense_msgs)
    return dense_msgs, num_timesteps, mmsis


dirname = os.path.dirname(dataset_path)

try:
    with tf.gfile.Open(dataset_path, "rb") as f:
        Vs = pickle.load(f)
except:
    with tf.gfile.Open(dataset_path, "rb") as f:
        Vs = pickle.load(f, encoding="latin1")

data_dim = LAT_BINS + LON_BINS + SOG_BINS + COG_BINS

mean_all = np.zeros((data_dim,))
sum_all = np.zeros((data_dim,))
total_ais_msg = 0

current_mean = np.zeros((0, data_dim))
current_ais_msg = 0

count = 0
for mmsi in list(Vs.keys()):
    count += 1
    print(count)
    tmp = Vs[mmsi][:, [LAT, LON, SOG, COG]]
    tmp[tmp == 1] = 0.99999
    current_sparse_matrix, _, _ = sparse_AIS_to_dense(tmp, 0, 0)
    #    current_mean = np.mean(current_sparse_matrix,axis = 0)
    sum_all += np.sum(current_sparse_matrix, axis=0)
    total_ais_msg += len(current_sparse_matrix)

mean = sum_all / total_ais_msg

print("Writing to", os.path.join(dirname, "/mean.pkl"))
with open(dirname + "/mean.pkl", "wb") as f:
    pickle.dump(mean, f)

### Training

#### Step 1: Training the Embedding layer

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import logging
import tensorflow as tf

import sys
sys.path.append("..")
import mobiml.models.geotracknet.runners as runners
from mobiml.models.geotracknet.flags_config import config

print(config.trainingset_path)
fh = logging.FileHandler(os.path.join(config.logdir, config.log_filename + ".log"))
tf.logging.set_verbosity(tf.logging.INFO)
# get TF logger
logger = logging.getLogger("tensorflow")
logger.addHandler(fh)
runners.run_train(config)

#### Step 2: Running task-specific submodels

In [ ]:
import glob
import pickle

with open(config.testset_path, "rb") as f:
    Vs_test = pickle.load(f)
dataset_size = len(Vs_test)


In [ ]:
step = None

tf.Graph().as_default()
global_step = tf.train.get_or_create_global_step()
inputs, targets, mmsis, time_starts, time_ends, lengths, model = (
    runners.create_dataset_and_model(config, shuffle=False, repeat=False)
)

if config.mode == "traj_reconstruction":
    config.missing_data = True

track_sample, track_true, log_weights, ll_per_t, ll_acc, _, _, _ = (
    runners.create_eval_graph(inputs, targets, lengths, model, config)
)
saver = tf.train.Saver()
sess = tf.train.SingularMonitoredSession()
runners.wait_for_checkpoint(saver, sess, config.logdir)
step = sess.run(global_step)

if step is None:
    # The log filename contains the step.
    index_filename = sorted(glob.glob(config.logdir+"/*.index"))[-1] # the lastest step
    step = int(index_filename.split(".index")[0].split("ckpt-")[-1])
    

print("Global step: ", step)
outputs_path = "results/"\
            + config.trainingset_path.split("/")[-2] + "/"\
            + "logprob-"\
            + os.path.basename(config.trainingset_name) + "-"\
            + os.path.basename(config.testset_name) + "-"\
            + str(config.latent_size)\
            + "-missing_data-" + str(config.missing_data)\
            + "-step-"+str(step)\
            + ".pkl"
if not os.path.exists(os.path.dirname(outputs_path)):
    os.makedirs(os.path.dirname(outputs_path))

save_dir = "results/"\
            + config.trainingset_path.split("/")[-2] + "/"\
            + "local_logprob-"\
            + os.path.basename(config.trainingset_name) + "-"\
            + os.path.basename(config.testset_name).replace("test","valid") + "-"\
            + str(config.latent_size) + "-"\
            + "missing_data-" + str(config.missing_data)\
            + "-step-"+str(step)\
            +"/"     

##### save_logprob

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
FIG_DPI = 150

l_dict = []
for d_i in tqdm(list(range(math.ceil(dataset_size / config.batch_size)))):
    inp, tar, mmsi, t_start, t_end, seq_len, log_weights_np, true_np, ll_t = (
        sess.run(
            [
                inputs,
                targets,
                mmsis,
                time_starts,
                time_ends,
                lengths,
                log_weights,
                track_true,
                ll_per_t,
            ]
        )
    )
    for d_idx_inbatch in range(inp.shape[1]):
        D = dict()
        seq_len_d = seq_len[d_idx_inbatch]
        D["seq"] = np.nonzero(tar[:seq_len_d, d_idx_inbatch, :])[1].reshape(-1, 4)
        D["t_start"] = t_start[d_idx_inbatch]
        D["t_end"] = t_end[d_idx_inbatch]
        D["mmsi"] = mmsi[d_idx_inbatch]
        D["log_weights"] = log_weights_np[:seq_len_d, :, d_idx_inbatch]
        l_dict.append(D)
with open(outputs_path, "wb") as f:
    pickle.dump(l_dict, f)

v_logprob = np.empty((0,))
v_logprob_stable = np.empty((0,))

count = 0
for D in tqdm(l_dict):
    log_weights_np = D["log_weights"]
    ll_t = np.mean(log_weights_np)
    v_logprob = np.concatenate((v_logprob, [ll_t]))

d_mean = np.mean(v_logprob)
d_std = np.std(v_logprob)
d_thresh = d_mean - 3 * d_std

plt.figure(figsize=(1920/FIG_DPI, 640/FIG_DPI), dpi=FIG_DPI)
plt.plot(v_logprob,'o')
plt.title("Log likelihood " + os.path.basename(config.testset_name)\
            + ", mean = {0:02f}, std = {1:02f}, threshold = {2:02f}".format(d_mean, d_std, d_thresh))
plt.plot([0,len(v_logprob)], [d_thresh, d_thresh],'r')

plt.xlim([0,len(v_logprob)])
fig_name = "results/"\
        + config.trainingset_path.split("/")[-2] + "/" \
        + "logprob-" \
        + config.bound + "-"\
        + os.path.basename(config.trainingset_name) + "-"\
        + os.path.basename(config.testset_name)\
        + "-latent_size-" + str(config.latent_size)\
        + "-ll_thresh" + str(round(d_thresh, 2))\
        + "-missing_data-" + str(config.missing_data)\
        + "-step-"+str(step)\
        + ".png"
plt.savefig(fig_name,dpi = FIG_DPI)
plt.close()

![](fig_name)

In [ ]:
from IPython.display import Image
Image(filename=fig_name) 

##### local_logprob

In [ ]:
import mobiml.models.geotracknet.utils as utils

LOGPROB_MEAN_MIN = -10.0
LOGPROB_STD_MAX = 5

LAT_RANGE = config.lat_max - config.lat_min
LON_RANGE = config.lon_max - config.lon_min
FIG_W = 960
FIG_H = int(FIG_W*LAT_RANGE/LON_RANGE)

m_map_logprob_std = np.zeros(shape=(config.n_lat_cells,config.n_lon_cells))
m_map_logprob_mean = np.zeros(shape=(config.n_lat_cells,config.n_lon_cells))
m_map_density = np.zeros(shape=(config.n_lat_cells,config.n_lon_cells))
v_logprob = np.empty((0,))
v_mmsi = np.empty((0,))
Map_logprob = dict()
for row  in range(config.n_lat_cells):
    for col in range(config.n_lon_cells):
        Map_logprob[ str(str(row)+","+str(col))] = []

# Load logprob
with open(outputs_path,"rb") as f:
    l_dict = pickle.load(f)

print("Calculating the logprob map...")
for D in tqdm(l_dict):
    tmp = D["seq"]
    log_weights_np = D["log_weights"]
    for d_timestep in range(2*6,len(tmp)):
        try:
            row = int(tmp[d_timestep,0]*0.01/config.cell_lat_reso)
            col = int((tmp[d_timestep,1]-config.onehot_lat_bins)*0.01/config.cell_lat_reso)
            Map_logprob[str(row)+","+str(col)].append(np.mean(log_weights_np[d_timestep,:]))
        except:
            continue

# Remove outliers
for row  in range(config.n_lat_cells):
    for col in range(config.n_lon_cells):
        s_key = str(row)+","+str(col) 
        Map_logprob[s_key] = utils.remove_gaussian_outlier(np.array(Map_logprob[s_key]))
        m_map_logprob_mean[row,col] = np.mean(Map_logprob[s_key])
        m_map_logprob_std[row,col] = np.std(Map_logprob[s_key])
        m_map_density[row,col] = len(Map_logprob[s_key])

# Save to disk
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
np.save(save_dir+"map_density-"+str(config.cell_lat_reso)+"-"+str(config.cell_lon_reso),m_map_density)
with open(os.path.join(save_dir,"Map_logprob-"+str(config.cell_lat_reso)+"-"+str(config.cell_lon_reso)+".pkl"),"wb") as f:
    pickle.dump(Map_logprob,f)

# Show the map
utils.show_logprob_map(m_map_logprob_mean, m_map_logprob_std, save_dir, 
                        logprob_mean_min = LOGPROB_MEAN_MIN,
                        logprob_std_max = LOGPROB_STD_MAX,
                        fig_w = FIG_W, fig_h = FIG_H,
                        )    

print(f'Maps stored saved to: {os.path.join(save_dir, "logprob_std_map.png")}')

In [ ]:
Image(filename=os.path.join(save_dir, "logprob_std_map.png")) 

##### contrario_detection

In [ ]:
import csv
from scipy import stats
from datetime import datetime
import mobiml.models.geotracknet.contrario_utils as contrario_utils

with open(os.path.join(save_dir,"Map_logprob-"+\
            str(config.cell_lat_reso)+"-"+str(config.cell_lat_reso)+".pkl"),"rb") as f:
    Map_logprob = pickle.load(f)
# Load the logprob
with open(outputs_path,"rb") as f:
    l_dict = pickle.load(f)
d_i = 0
v_mean_log = []
l_v_A = []
v_buffer_count = []
length_track = len(l_dict[0]["seq"])
l_dict_anomaly = []
n_error = 0
for D in tqdm(l_dict):
    try:
    # if True:
        tmp = D["seq"]
        m_log_weights_np = D["log_weights"]
        v_A = np.zeros(len(tmp))
        for d_timestep in range(2*6,len(tmp)):
            d_row = int(tmp[d_timestep,0]*config.onehot_lat_reso/config.cell_lat_reso)
            d_col = int((tmp[d_timestep,1]-config.onehot_lat_bins)*config.onehot_lat_reso/config.cell_lon_reso)
            d_logprob_t = np.mean(m_log_weights_np[d_timestep,:])

            # KDE
            l_local_log_prod = Map_logprob[str(d_row)+","+str(d_col)]
            if len(l_local_log_prod) < 2:
                v_A[d_timestep] = 2
            else:
                kernel = stats.gaussian_kde(l_local_log_prod)
                cdf = kernel.integrate_box_1d(-np.inf,d_logprob_t)
                if cdf < 0.1:
                    v_A[d_timestep] = 1
        v_A = v_A[12:]
        v_anomalies = np.zeros(len(v_A))
        for d_i_4h in range(0,len(v_A)+1-24):
            v_A_4h = v_A[d_i_4h:d_i_4h+24]
            v_anomalies_i = contrario_utils.contrario_detection(v_A_4h,config.contrario_eps)
            v_anomalies[d_i_4h:d_i_4h+24][v_anomalies_i==1] = 1

        if len(contrario_utils.nonzero_segments(v_anomalies)) > 0:
            D["anomaly_idx"] = v_anomalies
            l_dict_anomaly.append(D)
    except:
        n_error += 1
print("Number of processed tracks: ",len(l_dict))
print("Number of abnormal tracks: ",len(l_dict_anomaly)) 
print("Number of errors: ",n_error)

# Save to disk
n_anomalies = len(l_dict_anomaly)
save_filename = os.path.basename(config.trainingset_name)\
                +"-" + os.path.basename(config.trainingset_name)\
                +"-" + str(config.latent_size)\
                +"-missing_data-"+str(config.missing_data)\
                +"-step-"+str(step)\
                +".pkl"
save_pkl_filename = os.path.join(save_dir,"List_abnormal_tracks-"+save_filename)
with open(save_pkl_filename,"wb") as f:
    pickle.dump(l_dict_anomaly,f)

## Plot
with open(config.trainingset_path,"rb") as f:
    Vs_train = pickle.load(f)
with open(config.testset_path,"rb") as f:
    Vs_test = pickle.load(f)

save_filename = "Abnormal_tracks"\
            + "-" + os.path.basename(config.trainingset_name)\
            + "-" + os.path.basename(config.testset_name)\
            + "-latent_size-" + str(config.latent_size)\
            + "-step-"+str(step)\
            + "-eps-"+str(config.contrario_eps)\
            + "-" + str(n_anomalies)\
            + ".png"

# Plot abnormal tracks with the tracks in the training set as the background
utils.plot_abnormal_tracks(Vs_train,l_dict_anomaly,
                    os.path.join(save_dir,save_filename),
                    config.lat_min,config.lat_max,config.lon_min,config.lon_max,
                    config.onehot_lat_bins,config.onehot_lon_bins,
                    background_cmap = "Blues",
                    fig_w = FIG_W, fig_h = FIG_H,
                )
plt.close()
# Plot abnormal tracks with the tracks in the test set as the background
utils.plot_abnormal_tracks(Vs_test,l_dict_anomaly,
                    os.path.join(save_dir,save_filename.replace("Abnormal_tracks","Abnormal_tracks2")),
                    config.lat_min,config.lat_max,config.lon_min,config.lon_max,
                    config.onehot_lat_bins,config.onehot_lon_bins,
                    background_cmap = "Greens",
                    fig_w = FIG_W, fig_h = FIG_H,
                )
plt.close()   
# Save abnormal tracks to csv file
with open(os.path.join(save_dir,save_filename.replace(".png",".csv")),"w") as f:
    writer = csv.writer(f)
    writer.writerow(["MMSI","Time_start","Time_end","Timestamp_start","Timestamp_end"])
    for D in l_dict_anomaly:
        writer.writerow([D["mmsi"],
            datetime.utcfromtimestamp(D["t_start"]).strftime('%Y-%m-%d %H:%M:%SZ'),
            datetime.utcfromtimestamp(D["t_end"]).strftime('%Y-%m-%d %H:%M:%SZ'),
            D["t_start"],D["t_end"]])
        
print(f'Maps stored saved to: {save_dir}')

In [ ]:
Image(filename=os.path.join(save_dir,save_filename.replace("Abnormal_tracks","Abnormal_tracks2"))) 